<a href="https://colab.research.google.com/github/kazarezova/bioinformatics_for_NGS_exome/blob/main/kazarezova_ngs_exome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 
Используйте современный germline variant caller (в качестве основного будем
считать GATK, но вы можете выбрать другой) для получения стартового vcf файла.
Оцените полученные результаты:
- Количество синонимичных и несинонимичных замен
- ti/tv (отношение транзиций к трансверсиям) в кодирующих участках

In [ ]:
for i in ['chd_child.bam',
          'chd_child.bam.bai',
          'GRCh38_full_analysis_set_plus_decoy_hla.fa.gz',
          'GRCh38_full_analysis_set_plus_decoy_hla.fa.fai',
          'GRCh38_full_analysis_set_plus_decoy_hla.dict']:
          !wget https://makarich.fbb.msu.ru/enabieva/ngs2022/{i}

In [2]:
!gunzip GRCh38_full_analysis_set_plus_decoy_hla.fa.gz

### Variant caller

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda install -c bioconda gatk4

In [ ]:
!gatk --java-options "-Xmx4g" HaplotypeCaller -R GRCh38_full_analysis_set_plus_decoy_hla.fa -I chd_child.bam -O chd_child.vcf.gz

### Аннотируем варианты и оставляем только кодирующие мутации

In [ ]:
!wget https://snpeff.blob.core.windows.net/versions/snpEff_latest_core.zip
!unzip snpEff_latest_core.zip

In [7]:
!gunzip chd_child.vcf.gz

In [ ]:
!java -Xmx8g -jar snpEff/snpEff.jar GRCh38.99 chd_child.vcf > chd_child.ann.vcf

In [2]:
!egrep '#|synonymous|missense|stop_gained' chd_child.ann.vcf > chd_child.ann.coding.vcf

### Количество синонимичных и несинонимичных замен

In [3]:
!egrep '#|synonymous' chd_child.ann.coding.vcf | wc -l

14202


In [4]:
!egrep '#|missense|stop_gained' chd_child.ann.coding.vcf | wc -l

13716


### ti/tv (отношение транзиций к трансверсиям) в кодирующих участках



In [5]:
# https://pcingola.github.io/SnpEff/ss_tstv/
!java -jar snpEff/SnpSift.jar tstv chd_child.ann.coding.vcf

00:00:00 Analysing 'chd_child.ann.coding.vcf'
...................................................................................................
100	....................................................................................................
200	....................................................................................................
300	....................................................................................................
400	....................................................................................................
500	....................................................................................................
600	....................................................................................................
700	....................................................................................................
800	....................................................................................................
900	..........

# 2
Проведите фильтрацию полученных вариантов, используя либо
специализированные программы, либо “жесткую” фильтрацию (при которой вы
задаете пороги для разных свойств вариантов). Обоснуйте ваш выбор параметров
(следование рекомендациям - ок, но напишите, каким). Не забывайте, что
параметры могут быть разными для однонуклеотидных вариантов (SNPs) и
инделов.
Оцените количество синонимичных и несинонимичных замен и ts/tv после
фильтрации.
Прокомментируйте эффект фильтрации.


### Фильтрация SNP

In [ ]:
# https://gatk.broadinstitute.org/hc/en-us/articles/360035531112--How-to-Filter-variants-either-with-VQSR-or-by-hard-filtering#2
!gatk SelectVariants -V chd_child.vcf -select-type SNP -O chd_child.snps.vcf.gz

In [ ]:
# https://gatk.broadinstitute.org/hc/en-us/articles/360035531112--How-to-Filter-variants-either-with-VQSR-or-by-hard-filtering
!gatk VariantFiltration \
    -V chd_child.snps.vcf.gz \
    -filter "QD < 2.0" --filter-name "QD2" \
    -filter "QUAL < 30.0" --filter-name "QUAL30" \
    -filter "SOR > 3.0" --filter-name "SOR3" \
    -filter "FS > 60.0" --filter-name "FS60" \
    -filter "MQ < 40.0" --filter-name "MQ40" \
    -filter "MQRankSum < -12.5" --filter-name "MQRankSum-12.5" \
    -filter "ReadPosRankSum < -8.0" --filter-name "ReadPosRankSum-8" \
    -O chd_child.snps_filtered.vcf.gz

In [ ]:
!gunzip chd_child.snps_filtered.vcf.gz
!java -Xmx8g -jar snpEff/snpEff.jar GRCh38.99 chd_child.snps_filtered.vcf > chd_child.snps_filtered.ann.vcf
!egrep '#|synonymous|missense|stop_gained' chd_child.snps_filtered.ann.vcf > chd_child.snps_filtered.ann.coding.vcf

In [18]:
!grep 'PASS' chd_child.snps_filtered.ann.coding.vcf > chd_child.snps_filtered_pass.ann.coding.vcf

### Количество синонимичных и несинонимичных замен после фильтрации SNP

In [19]:
!egrep '#|synonymous' chd_child.snps_filtered_pass.ann.coding.vcf | wc -l # стало на 3710 меньше

10492


In [20]:
!egrep '#|missense|stop_gained' chd_child.snps_filtered_pass.ann.coding.vcf | wc -l # стало на 3789 меньше

9927


### ti/tv в кодирующих участках после фильтрации

In [21]:
!java -jar snpEff/SnpSift.jar tstv chd_child.snps_filtered_pass.ann.coding.vcf # было 3.055, стало 3.071 - слабо изменилось

00:00:00 Analysing 'chd_child.snps_filtered_pass.ann.coding.vcf'
...................................................................................................
100	....................................................................................................
200	....................................................................................................
300	....................................................................................................
400	....................................................................................................
500	....................................................................................................
600	....................................................................................................
700	....................................................................................................
800	................................................................................................

# 3 
Напишите, сколько в вашем окончательном списке:
- Protein truncating variants (инделов со сдвигом рамки считывания,
nonsense-вариантов)
- Гомозиготных вариантов, изменяющих последовательность белка
- Приведите их список

### Protein truncating variants

In [22]:
# nonsense-варианты

!egrep '#|stop_gained' chd_child.snps_filtered_pass.ann.coding.vcf | wc -l

89


In [73]:
!egrep '#|stop_gained' chd_child.snps_filtered_pass.ann.coding.vcf > prot_trunc_nonsense.tsv

In [78]:
import pandas as pd

df = pd.DataFrame(pd.read_csv('prot_trunc_nonsense.tsv', sep='\t', names=['CHROM',	'POS',	'ID',	'REF',	'ALT',	'QUAL',	'FILTER',	'INFO',	'FORMAT',	'justchild']))
df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,justchild
0,chr1,40307478,.,G,A,142.64,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=-0.987;DP=11;E...,GT:AD:DP:GQ:PL,"0/1:6,4:10:99:150,0,240"
1,chr1,46615007,.,G,A,447.64,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=-2.380;DP=34;E...,GT:AD:DP:GQ:PL,"0/1:13,18:31:99:455,0,366"
2,chr1,171208951,.,C,T,385.06,PASS,AC=2;AF=1.00;AN=2;DP=13;ExcessHet=0.0000;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,13:13:39:399,39,0"
3,chr1,228282202,.,A,T,109.64,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=-1.036;DP=6;Ex...,GT:AD:DP:GQ:PL,"0/1:3,3:6:99:117,0,117"
4,chr1,247949724,.,T,A,311.06,PASS,AC=2;AF=1.00;AN=2;DP=10;ExcessHet=0.0000;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,10:10:30:325,30,0"
...,...,...,...,...,...,...,...,...,...,...
84,chr22,16988159,.,C,A,204.04,PASS,AC=2;AF=1.00;AN=2;DP=8;ExcessHet=0.0000;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,8:8:24:218,24,0"
85,chr22,26466075,.,G,A,188.64,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=0.488;DP=27;Ex...,GT:AD:DP:GQ:PL,"0/1:18,8:26:99:196,0,494"
86,chrX,35803010,.,C,T,274.64,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=0.348;DP=24;Ex...,GT:AD:DP:GQ:PL,"0/1:12,11:23:99:282,0,307"
87,chrX,75784694,.,C,A,475.06,PASS,AC=2;AF=1.00;AN=2;DP=16;ExcessHet=0.0000;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,15:15:45:489,45,0"


In [ ]:
# инделы со сдвигом рамки считывания (т.е. любые кодирующие инделы, кроме синонимичных)

!gatk SelectVariants -V chd_child.vcf -select-type INDEL -O chd_child.indels.vcf.gz

!gatk VariantFiltration -V chd_child.indels.vcf.gz -filter "QD < 2.0" --filter-name "QD2" -filter "QUAL < 30.0" --filter-name "QUAL30" -filter "FS > 200.0" --filter-name "FS200" -filter "ReadPosRankSum < -20.0" --filter-name "ReadPosRankSum-20" -O chd_child.indels_filtered.vcf.gz

!gunzip chd_child.indels_filtered.vcf.gz
!java -Xmx8g -jar snpEff/snpEff.jar GRCh38.99 chd_child.indels_filtered.vcf > chd_child.indels_filtered.ann.vcf
!egrep '#|missense|stop_gained' chd_child.indels_filtered.ann.vcf > chd_child.indels_filtered.ann.coding.vcf

In [34]:
!grep 'PASS' chd_child.indels_filtered.ann.coding.vcf | wc -l

5


In [75]:
!grep 'PASS' chd_child.indels_filtered.ann.coding.vcf > prot_trunc_indels.tsv

In [80]:
df = pd.DataFrame(pd.read_csv('prot_trunc_indels.tsv', sep='\t', names=['CHROM',	'POS',	'ID',	'REF',	'ALT',	'QUAL',	'FILTER',	'INFO',	'FORMAT',	'justchild']))
df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,justchild
0,chr3,149520808,.,C,CTTAA,490.60,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=1.697;DP=21;Ex...,GT:AD:DP:GQ:PL,"0/1:8,13:21:99:498,0,297"
1,chr7,2513247,.,A,AGATG,1014.03,PASS,AC=2;AF=1.00;AN=2;DP=26;ExcessHet=0.0000;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,24:24:72:1028,72,0"
2,chr7,152247986,.,G,GT,881.60,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=0.006;DP=75;Ex...,GT:AD:DP:GQ:PL,"0/1:43,30:73:99:889,0,1391"
3,chr19,52374463,.,A,AT,45.60,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=0.417;DP=24;Ex...,GT:AD:DP:GQ:PL,"0/1:11,4:15:53:53,0,187"
4,chrX,36144567,.,C,CTG,94.60,PASS,AC=1;AF=0.500;AN=2;BaseQRankSum=0.728;DP=12;Ex...,GT:AD:DP:GQ:PL,"0/1:8,3:11:99:102,0,306"


### Гомозиготные варианты, изменяющие последовательность белка

In [98]:
# гомозиготные HO: Indicated homozygous variants (i.e. '1/1') http://pcingola.github.io/SnpEff/ss_gt/
!grep '1/1:' chd_child.snps_filtered_pass.ann.coding.vcf > chd_child.snps_filtered_pass.homo.ann.coding.vcf

In [100]:
# изменяющие последовательность - missense

!egrep '#|missense' chd_child.snps_filtered_pass.homo.ann.coding.vcf | wc -l

4381


In [101]:
!egrep '#|missense' chd_child.snps_filtered_pass.homo.ann.coding.vcf > homo_missense.tsv
df = pd.DataFrame(pd.read_csv('homo_missense.tsv', sep='\t', names=['CHROM',	'POS',	'ID',	'REF',	'ALT',	'QUAL',	'FILTER',	'INFO',	'FORMAT',	'justchild']))
df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,justchild
0,chr1,69511,.,A,G,98.14,PASS,AC=2;AF=1.00;AN=2;DP=5;ExcessHet=0.0000;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,4:4:12:112,12,0"
1,chr1,953279,.,T,C,713.06,PASS,AC=2;AF=1.00;AN=2;DP=26;ExcessHet=0.0000;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,26:26:78:727,78,0"
2,chr1,976215,.,A,G,396.06,PASS,AC=2;AF=1.00;AN=2;DP=16;ExcessHet=0.0000;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,15:15:45:410,45,0"
3,chr1,978953,.,C,G,193.97,PASS,AC=2;AF=1.00;AN=2;DP=6;ExcessHet=0.0000;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,6:6:18:208,18,0"
4,chr1,979472,.,G,C,706.06,PASS,AC=2;AF=1.00;AN=2;DP=16;ExcessHet=0.0000;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,16:16:48:720,48,0"
...,...,...,...,...,...,...,...,...,...,...
4376,chrX,153774047,.,G,C,222.04,PASS,AC=2;AF=1.00;AN=2;DP=9;ExcessHet=0.0000;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,8:8:24:236,24,0"
4377,chrX,153776089,.,T,C,194.97,PASS,AC=2;AF=1.00;AN=2;DP=8;ExcessHet=0.0000;FS=0.0...,GT:AD:DP:GQ:PL,"1/1:0,6:6:18:209,18,0"
4378,chrX,154465991,.,G,C,385.06,PASS,AC=2;AF=1.00;AN=2;DP=13;ExcessHet=0.0000;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,13:13:39:399,39,0"
4379,chrX,154485448,.,T,C,462.06,PASS,AC=2;AF=1.00;AN=2;DP=17;ExcessHet=0.0000;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,17:17:51:476,51,0"
